# ДЗ к вебинару 2

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекомендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

Сформируем items_weights: pd.DataFrame
Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1

In [60]:
import pandas as pd
import numpy as np

In [61]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [62]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [63]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [64]:
# Просуммируем sales_value для каждого item_id
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights.head(3)

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59


In [65]:
# Нормируем sales_value так, чтобы получить столбец weights с суммой 1 по всем товарам. Столбец sales_value после этого можно удалить.
items_weights['weights'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights.drop(labels='sales_value', axis=1, inplace=True)
items_weights.head(3)

,item_id,weights
0,25671,2.969296e-06
1,26081,1.403822e-07
2,26093,2.254623e-07


In [66]:
def weighted_random_recommendation(items_weights, n=5):
    """    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weights'])
    
    return recs.tolist()

In [67]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

Wall time: 14.9 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[16809278, 943940, 994928, 985999, 1012851]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[939831, 1091365, 1010076, 9365871, 6534178]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [68]:
result1 = pd.read_csv('output/predictions_basic.csv')
result1.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1381582, 870101, 32805, 80472, 8203764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[636668, 856207, 516046, 863639, 1102674]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


Значения в колонках с предсказаниями и выбором пользователя имеют тип str. Необходимо преобразовать их к list, чтобы в дальнейшем не было проблем с их обработкой при подсчете precision. Для этого напишем и применим к колонкам функцию преобразования строки.

In [69]:
def str_to_list(my_str):
    if type(my_str) is str:
        return my_str.replace(",", " ").strip('[').strip(']').split()
    else:
        return my_str

Будем пытаться получить усредненное значение метрики Precision@5 по всем пользователям. Для этого воспользуемся уже готовой функцией precision_at_k.

In [70]:
result1 = result1.applymap(lambda x: str_to_list(x))
result1.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1381582, 870101, 32805, 80472, 8203764]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[636668, 856207, 516046, 863639, 1102674]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [71]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [72]:
def mean_p5(recommended_list_list, bought_list_list, k=5): # recommended_list_list и bought_list_list - в виде pandas series, содержащих списки идентификаторов
    
    p5 = [precision_at_k(recommended_list_list[i], bought_list_list[i], k=5) for i in range(len(recommended_list_list))]
        
    return sum(p5)/len(p5)

In [73]:
mp5 = pd.DataFrame(columns=['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 'own_purchases'])

In [76]:
mp5.loc['k5'] = [mean_p5(result1[alg], result1['actual'], k=5) for alg in mp5.columns]
mp5

,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
k5,0.000196,0.15524,0.136925,0.132909,0.138981,0.179693


Таким образом, можно сделать вывод, что наилучший результат дает метод Item-Item Recommender с одним ближайшим соседом (столбец own_purchases). Чуть хуже - метод с подбором популярных товаров (столбец popular_recommendation). Самые плохие результаты (сильно хуже остальных) дает метод случайных рекомендаций (столбец random_recommendation).